This code is a result of a recent research on portfolio theory. One can see the paper

 ***Halidias, Nikolaos. "A novel portfolio optimization method and its application to the hedging problem" Monte Carlo Methods and Applications, vol. 30, no. 3, 2024, pp. 249-267,*** and the references therein. (https://www.degruyterbrill.com/document/doi/10.1515/mcma-2024-2009/html)

See also the book
***Halidias, Nikolaos and Stamatiou, Ioannis S.. Stochastic Analysis: Financial Mathematics with Matlab®, De Gruyter, 2026.*** (https://www.degruyterbrill.com/document/isbn/9783111443737/html?lang=en&srsltid=AfmBOorukbLCPRdRPn2Mu2kqSWiGttsEAPF6wBfVNixx4fAFzxycr6Fl)

At the webpage of the ***Actuarial-Financial Laboratory*** https://www.samos.aegean.gr/actuar/nick/actfinlabeng.htm one can find also other useful informations.



We use all the call and put options from the yahoo finace in order to construct the portfolio. If you choose experiment=1 then we use the lastPrices otherwise the bid-ask prices. If for any reason you want to take account only some of the call and put options you should manually set the data stock_value, call_strikes,put_strikes,call_ask,call_bid,put_ask,put_bid.



The payoff of the option should have finite number of branches where the last branch should be a linear function. You should find the slope of the last branch and all the local extrema.


We compute the set of arbitrage free prices and the fair and arbitrage free price $Y^{D^*}$ as this has been proposed at the above paper. Though we can propose several fair prices this fair price is the only one which is also arbitrage free for sure. Therefore this price can be seen as the unique fair and arbitrage free price in some sense.

In [4]:
import numpy as np
import yfinance as yf

# Ζητάμε από τον χρήστη να εισάγει το ticker
Firm = input("Εισάγετε το ticker symbol (π.χ. AAPL): ").strip().upper()

# Φορτώνουμε το asset
stock = yf.Ticker(Firm)

# Παίρνουμε τις διαθέσιμες expiration dates
expirations = stock.options

if not expirations:
    print(f"Δεν βρέθηκαν expiration dates για {Firm}")
else:
    print("\nΔιαθέσιμες Expiration Dates:")
    for i, date in enumerate(expirations):
        print(f"{i+1}. {date}")

    # Ζητάμε να επιλέξει ημερομηνία και προσθέτουμε έλεγχο
    while True:
        choice_str = input("\nΕπιλέξτε αριθμό expiration date: ").strip()
        try:
            choice = int(choice_str) - 1
            if 0 <= choice < len(expirations):
                ExpirationDate = expirations[choice]
                print(f"\nΦορτώνω δεδομένα για {Firm} με expiration date {ExpirationDate}...")
                break # Exit the loop if input is valid
            else:
                print("Μη έγκυρη επιλογή. Παρακαλώ εισάγετε έναν αριθμό από τη λίστα.")
        except ValueError:
            print("Μη έγκυρη εισαγωγή. Παρακαλώ εισάγετε έναν αριθμό.")


#############################################################################################################
# You should set the following data
Data_given=1 # Set 1 if you want to download data from Yahoo finance otherwise read data from excel file
#Firm='HTZ'
#ExpirationDate='2025-07-18'
excel_file='excel_file3.xlsx'
experiment=0 # Set 1 if you want to experiment. Then the lastPrices will be used otherwise the bid-ask prices
max_number_of_total_call_options_to_buy=40 # Maximum number of total call options  to buy
max_number_of_total_put_options_to_buy=40 # Maximum number of total put options to buy
max_number_of_total_call_options_to_sell=40 # Maximum number of total call options to sell
max_number_of_total_put_options_to_sell=40 # Maximum number of total put options to sell
max_number_of_call_options_to_buy=1  # the number of maximum number of call options of each strike to buy
max_number_of_put_options_to_buy=1 # the number of maximum number of put options of each strike to buy
max_number_of_call_options_to_sell=1 # the number of maximum number of call options of each strike to sell
max_number_of_put_options_to_sell=1 # the number of maximum number of put options of each strike to sell
max_number_of_shares_to_buy=100
max_number_of_shares_to_sell=100
max_amount_to_bank_account=100
max_amount_from_bank_account=100
#$$$$$$$$$$$$$$$$$$$$$$$$
# Give the following information for the new option
def payoff_function(x):
    return np.maximum(np.maximum(x-5,0),np.maximum(3-x,0))
derivative_of_payoff=1 # The slope of the last branch
payoff_nodes=np.array([[0,3]]) # Give here the points of local extrema
#$$$$$$$$$$$$$$$$$$$$$$
# If you can not find by yourself the slope of the last branch and the payoff_nodes you can use AI
# For example you can ask AI: Determine  the slope of the last branch and the local extrema  of the function def payoff_function(x):
# return np.minimum(18,np.maximum(x-2.9,0))
###################################################################################################################



#######################################################################################################################
# Here is the function that download the data from yahoo finance
def options_bid_ask(Firm,ExpirationDate,experiment):
    import numpy as np
    import yfinance as yt
    stock = yt.Ticker(Firm)
    stock_value = stock.history(period='1d')['Close'].iloc[-1]
    options_chain = stock.option_chain(ExpirationDate)
    call_strikes=options_chain.calls.strike.to_numpy()
    put_strikes=options_chain.puts.strike.to_numpy()
    if experiment==1:
       call_ask = options_chain.calls.lastPrice.to_numpy()
       call_bid=options_chain.calls.lastPrice.to_numpy()
       put_ask=options_chain.puts.lastPrice.to_numpy()
       put_bid=options_chain.puts.lastPrice.to_numpy()
    else:
       call_ask = options_chain.calls.ask.to_numpy()
       call_bid=options_chain.calls.bid.to_numpy()
       put_ask=options_chain.puts.ask.to_numpy()
       put_bid=options_chain.puts.bid.to_numpy()
    return stock_value, call_strikes,put_strikes,call_ask,call_bid,put_ask,put_bid
############################################################################################################


##################################################################################################
# Here is the function that take the data from the excel file.
def options_bid_ask_excel(experiment):
    import pandas as pd
    import numpy as np
    try:
        df = pd.read_excel(excel_file)
        stock_value = df.iloc[0, 0]
        call_strikes = df.iloc[:, 1].dropna().to_numpy()
        put_strikes = df.iloc[:, 5].dropna().to_numpy()
        if experiment==1:
            call_ask = df.iloc[:, 2].dropna().to_numpy()
            call_bid = df.iloc[:, 2].dropna().to_numpy()
            put_ask = df.iloc[:, 6].dropna().to_numpy()
            put_bid = df.iloc[:, 6].dropna().to_numpy()
        else:
            call_ask = df.iloc[:, 4].dropna().to_numpy()
            call_bid = df.iloc[:, 3].dropna().to_numpy()
            put_ask = df.iloc[:, 8].dropna().to_numpy()
            put_bid = df.iloc[:, 7].dropna().to_numpy()
        return stock_value, call_strikes,put_strikes,call_ask,call_bid,put_ask,put_bid
    except FileNotFoundError: # Added a basic except block to handle file not found errors
        print("Error: 'excel_file.xlsx' not found. Please make sure the file exists in the current directory.")
        return None, None, None, None, None, None, None # Return None values if file not found
####################################################################################################################






##########################################################################################
# Here we choose how we collect the data
if Data_given==1:
   stock_value, call_strikes,put_strikes,call_ask1,call_bid,put_ask1,put_bid=options_bid_ask(Firm,ExpirationDate,experiment)
else:
   stock_value, call_strikes,put_strikes,call_ask1,call_bid,put_ask1,put_bid=options_bid_ask_excel(experiment)
###########################################################################################

#####################################################################################
# We will remove zeros from the ask prices
keep_mask_call = call_ask1 != 0
keep_mask_put = put_ask1 !=0
call_ask = call_ask1[keep_mask_call]
call_bid = call_bid[keep_mask_call]
put_ask = put_ask1[keep_mask_put]
put_bid = put_bid[keep_mask_put]
call_strikes=call_strikes[keep_mask_call]
put_strikes=put_strikes[keep_mask_put]
#######################################################################################


###############################################################################################
union_strikes1=np.union1d(call_strikes,put_strikes) #The union of call_strikes and put_strikes
union_strikes=np.union1d(union_strikes1,payoff_nodes) # We add the payoff nodes
max_element = union_strikes.max() # We need this for the plot
##########################################################################################



#############################################################################################
# This will give the row of the matrix A for each strike price.
# The first element is the number of shares, next bank account, next call options to buy,
# call options to sell, put options to buy, put options to sell and the maximum loss
def PP(x):
    matrixrow = np.maximum(x - call_strikes, 0) #call to buy
    matrixrow = np.append(matrixrow, -np.maximum(x-call_strikes, 0)) # call to sell
    matrixrow=np.append(matrixrow, np.maximum(put_strikes-x, 0)) # put to buy
    matrixrow=np.append(matrixrow, -np.maximum(put_strikes-x, 0)) # put to sell
    matrixrow=np.append(1,matrixrow) # bank account
    matrixrow = np.append(x, matrixrow) # shares
    matrixrow = np.append(matrixrow, 1) # maximum loss
    return matrixrow
###############################################################################################



###########################################################################################
# We begin to set the matrix AA
AA = PP(0)
number_of_columns_AA = AA.shape[0]
for x in union_strikes:
    AA=np.vstack([AA, PP(x)])
AA = -AA
###########################################################################################


############################################################################################
# Adding the row with the derivative
Aderiv = np.zeros(number_of_columns_AA)
for i in range(1,len(call_strikes)+2):
    Aderiv[i] = -1
for i in range(len(call_strikes)+2, 2*len(call_strikes)+2):
    Aderiv[i] = 1
Aderiv[1]=0
Aderiv[0]=-1
AA = np.vstack([AA, Aderiv])
############################################################################################






#########################################################################################
# Adding the row with options prices
Aequality=call_ask
Aequality=np.append(Aequality,-call_bid)
Aequality=np.append(Aequality,put_ask)
Aequality=np.append(Aequality,-put_bid)
Aequality=np.append(1,Aequality) # Bank account
Aequality=np.append(stock_value,Aequality)
Aequality=np.append(Aequality,0)
AA=np.vstack([AA,Aequality])
#########################################################################################



########################################################################################
# Adding the rows for the maximum number of options
Amaxnumberoptions_call_buy=np.full(number_of_columns_AA,0)
Amxanumberoptions_call_sell=np.full(number_of_columns_AA,0)
Amaxnumberoptions_put_buy=np.full(number_of_columns_AA,0)
Amaxnumberoptions_put_sell=np.full(number_of_columns_AA,0)
for i in range(2,len(call_strikes)+2):
    Amaxnumberoptions_call_buy[i]=1
for i in range(len(call_strikes)+2,2*len(call_strikes)+2):
    Amxanumberoptions_call_sell[i]=1
for i in range(2*len(call_strikes)+2,2*len(call_strikes)+len(put_strikes)+2):
    Amaxnumberoptions_put_buy[i]=1
for i in range(2*len(call_strikes)+len(put_strikes)+2,2*len(call_strikes)+2*len(put_strikes)+2):
    Amaxnumberoptions_put_sell[i]=1
AA=np.vstack([AA,Amaxnumberoptions_call_buy])
AA=np.vstack([AA,Amxanumberoptions_call_sell])
AA=np.vstack([AA,Amaxnumberoptions_put_buy])
AA=np.vstack([AA,Amaxnumberoptions_put_sell])
#############################################################################################



#######################################################################
# Setting the upper bounds for the parameters
ub=np.full(AA.shape[1], 0, dtype=float)
ub[0]=max_number_of_shares_to_buy
ub[1]=max_amount_to_bank_account # Bank account
for i in range(2,len(call_strikes)+2):
        ub[i]=max_number_of_call_options_to_buy
for i in range(2*len(call_strikes)+2,2*len(call_strikes)+len(put_strikes)+2):
        ub[i]=max_number_of_put_options_to_buy
for i in range(len(call_strikes)+2,2*len(call_strikes)+2):
        ub[i]=max_number_of_call_options_to_sell
for i in range(2*len(call_strikes)+len(put_strikes)+2,2*len(call_strikes)+2*len(put_strikes)+2):
        ub[i]=max_number_of_put_options_to_sell
ub[-1]=np.inf
########################################################################

###################################################################
# Setting the lower bounds for the parameters
lb=np.full(AA.shape[1], 0, dtype=float)
lb[0]=-max_number_of_shares_to_sell
lb[-1]=-np.inf
lb[1]=-max_amount_from_bank_account # bank account
###################################################################

###########################################################################
# We assume that we can buy  only integer number of options
integrality=np.full(AA.shape[1],1)
integrality[0]=0 # 1 for integer number of shares, 0 for fractional number of shares
integrality[-1]=0
integrality[1]=0 # bank account
############################################################################

#####################################################################
# We set the quantity to be minimized
f=np.zeros(AA.shape[1])
f[-1]=1
#######################################################################




##########################################################################################################
def FairValue(fair_value):
    ######################################################################################
    # Setting the right hand side for the writer
    bw = np.zeros(AA.shape[0])
    bw[0]=-payoff_function(0)
    for i in range(1,len(union_strikes)+1):
       bw[i] =  - payoff_function(union_strikes[i-1]) # For the writer this should be (-)
    bw[-4]=max_number_of_total_call_options_to_buy
    bw[-3]=max_number_of_total_call_options_to_sell
    bw[-2]=max_number_of_total_put_options_to_buy
    bw[-1]=max_number_of_total_put_options_to_sell
    bw[len(union_strikes)+1]=-derivative_of_payoff # for the row with the derivatives
    bw[-5]=fair_value # for the total amount used
    #######################################################################################



    ####################################################################
    # Setting the left hand side for the writer
    blw=np.full_like(bw, -np.inf, dtype=float)
    blw[-5]=fair_value
    ####################################################################

    ######################################################################################
    # Setting the right hand side for the buyer
    bb = np.zeros(AA.shape[0])
    bb[0]=payoff_function(0)
    for i in range(1,len(union_strikes)+1):
       bb[i] =   payoff_function(union_strikes[i-1]) # For the writer this should be (-)
    bb[-4]=max_number_of_total_call_options_to_buy
    bb[-3]=max_number_of_total_call_options_to_sell
    bb[-2]=max_number_of_total_put_options_to_buy
    bb[-1]=max_number_of_total_put_options_to_sell
    bb[len(union_strikes)+1]=derivative_of_payoff # for the row with the derivatives
    bb[-5]=-fair_value # for the total amount used
    #######################################################################################



    ####################################################################
    # Setting the left hand side for the buyer
    blb=np.full_like(bb, -np.inf, dtype=float)
    blb[-5]=-fair_value
    ####################################################################

    from scipy.optimize import milp
    from scipy.optimize import LinearConstraint
    from scipy.optimize import Bounds
    ###################################################################
    # We define the constraints and the bounds for the writer
    constraints_writer=LinearConstraint(AA,blw,bw)
    bounds=Bounds(lb,ub)
    ##################################################################

    ###################################################################
    # We define the constraints and the bounds for the buyer
    constraints_buyer=LinearConstraint(AA,blb,bb)
    bounds=Bounds(lb,ub)
    ##################################################################

    #########################################################################
    # We solve the optimization problem for the writer
    result_writer = milp(c=f, constraints=constraints_writer, integrality=integrality, bounds=bounds)
    # We solve the optimization problem for the buyer
    result_buyer = milp(c=f, constraints=constraints_buyer, integrality=integrality, bounds=bounds)
    #########################################################################
    return result_writer.fun, result_buyer.fun
######################################################################################################






#########################################################################################################
def Y_buyer(fair_value):
    ######################################################################################
    # Setting the right hand side for the buyer
    bb = np.zeros(AA.shape[0])
    bb[0]=payoff_function(0)
    for i in range(1,len(union_strikes)+1):
       bb[i] =   payoff_function(union_strikes[i-1]) # For the writer this should be (-)
    bb[-4]=max_number_of_total_call_options_to_buy
    bb[-3]=max_number_of_total_call_options_to_sell
    bb[-2]=max_number_of_total_put_options_to_buy
    bb[-1]=max_number_of_total_put_options_to_sell
    bb[len(union_strikes)+1]=derivative_of_payoff # for the row with the derivatives
    bb[-5]=-fair_value # for the total amount used
    #######################################################################################



    ####################################################################
    # Setting the left hand side for the buyer
    blb=np.full_like(bb, -np.inf, dtype=float)
    blb[-5]=-fair_value
    ####################################################################

    from scipy.optimize import milp
    from scipy.optimize import LinearConstraint
    from scipy.optimize import Bounds


    ###################################################################
    # We define the constraints and the bounds for the buyer
    constraints_buyer=LinearConstraint(AA,blb,bb)
    bounds=Bounds(lb,ub)
    ##################################################################

    #########################################################################

    # We solve the optimization problem for the buyer
    result_buyer = milp(c=f, constraints=constraints_buyer, integrality=integrality, bounds=bounds)
    #########################################################################
    return  result_buyer.fun
######################################################################################################


######################################################################################################
def Y_writer(fair_value):
    ######################################################################################
    # Setting the right hand side for the writer
    bw = np.zeros(AA.shape[0])
    bw[0]=-payoff_function(0)
    for i in range(1,len(union_strikes)+1):
       bw[i] =  - payoff_function(union_strikes[i-1]) # For the writer this should be (-)
    bw[-4]=max_number_of_total_call_options_to_buy
    bw[-3]=max_number_of_total_call_options_to_sell
    bw[-2]=max_number_of_total_put_options_to_buy
    bw[-1]=max_number_of_total_put_options_to_sell
    bw[len(union_strikes)+1]=-derivative_of_payoff # for the row with the derivatives
    bw[-5]=fair_value # for the total amount used
    #######################################################################################


    ####################################################################
    # Setting the left hand side for the writer
    blw=np.full_like(bw, -np.inf, dtype=float)
    blw[-5]=fair_value
    ####################################################################


    from scipy.optimize import milp
    from scipy.optimize import LinearConstraint
    from scipy.optimize import Bounds
    ###################################################################
    # We define the constraints and the bounds for the writer
    constraints_writer=LinearConstraint(AA,blw,bw)
    bounds=Bounds(lb,ub)
    ##################################################################


    #########################################################################
    # We solve the optimization problem for the writer
    result_writer = milp(c=f, constraints=constraints_writer, integrality=integrality, bounds=bounds)
    #########################################################################
    return result_writer.fun
############################################################################################################




################################################################################################
# We compute the smallest  price of the option for which the writer can construct a prtofolio with zero maximum possible loss
y_writer=stock_value/2
max_possible_loss_writer=1
while np.abs(max_possible_loss_writer)>0.001:
      if max_possible_loss_writer>0:
            y_writer+=np.abs(max_possible_loss_writer)/2
      else:
            y_writer-=np.abs(max_possible_loss_writer)/2
      max_possible_loss_writer=Y_writer(y_writer)
####################################################################################################


#####################################################################################################
# We compute the maximum price of the option with which the buyer can construct a portfolio with zero maximum possible loss
y_buyer=y_writer
max_possible_loss_buyer=1
while np.abs(max_possible_loss_buyer)>0.001:
      if max_possible_loss_buyer>0:
            y_buyer-=np.abs(max_possible_loss_buyer)/2
      else:
            y_buyer+=np.abs(max_possible_loss_buyer)/2
      max_possible_loss_buyer=Y_buyer(y_buyer)
#############################################################################################################

#############################################################################################################
# We compute the fair value of the option
fair_value=(y_writer+y_buyer)/2
max_possible_loss_writer=1
max_possible_loss_buyer=0
while np.abs(max_possible_loss_writer-max_possible_loss_buyer)>0.001:
      if max_possible_loss_writer>max_possible_loss_buyer:
            fair_value+=np.abs(max_possible_loss_writer-max_possible_loss_buyer)/2
      else:
            if np.abs(max_possible_loss_writer-max_possible_loss_buyer)/2> fair_value:
               fair_value=fair_value/2
            else:
               fair_value-=np.abs(max_possible_loss_writer-max_possible_loss_buyer)/2

      max_possible_loss_writer, max_possible_loss_buyer=FairValue(fair_value)
#########################################################################################################






#################################################################################################################
# Printing the results
if y_buyer>y_writer:
   print('The market has arbitrage opportunities. That means that there is no arbitrage free price.')
   print('A fair price but not arbitrage free is ', fair_value)
else:
   print('The fair and arbitrage free price of the option equals to', fair_value)
   print('The arbitrage free interval is','(',y_buyer,',',y_writer,')')
   print('The center of the arbitrage free interval is', (y_buyer+y_writer)/2)
   print('Selling the option at the price', y_writer+0.0011, 'the maximum possible loss  for the writer is', Y_writer(y_writer+0.0011))
   print('Selling the option at the price', y_writer-0.0011, 'the maximum possible loss  for the writer is', Y_writer(y_writer-0.0011))
   print('Buying the option at the price', y_buyer-0.0011, 'the maximum possible loss for the buyer is', Y_buyer(y_buyer-0.0011))
   print('Buying the option at the price', y_buyer+0.0011, 'the maximum possible loss for the buyer is', Y_buyer(y_buyer+0.0011))
##################################################################################################################

Εισάγετε το ticker symbol (π.χ. AAPL): HTZ

Διαθέσιμες Expiration Dates:
1. 2025-06-06
2. 2025-06-13
3. 2025-06-20
4. 2025-06-27
5. 2025-07-03
6. 2025-07-11
7. 2025-07-18
8. 2025-09-19
9. 2025-10-17
10. 2025-12-19
11. 2026-01-16
12. 2026-06-18
13. 2026-12-18
14. 2027-01-15

Επιλέξτε αριθμό expiration date: 10

Φορτώνω δεδομένα για HTZ με expiration date 2025-12-19...
The market has arbitrage opportunities. That means that there is no arbitrage free price.
A fair price but not arbitrage free is  2.79999999999789
